In [13]:
import timed_nodes as tn

if __name__ == "__main__":
    input_file = "unspoiled_nodes"      
    output_file = "cleaned_nodes.csv"    
    tn.clean_unspoiled_data(input_file, output_file)
    print(f"Processed '{input_file}' and wrote results to '{output_file}'.")


Processed 'unspoiled_nodes' and wrote results to 'cleaned_nodes.csv'.


In [14]:
if __name__ == "__main__":
    input_file = "cleaned_nodes.csv"      
    output_file = "final_nodes_with_ids.csv"    
    tn.assign_ids(input_file, output_file)
    print(f"Merged CSV '{input_file}' and wrote results to '{output_file}'.")

Merged CSV written to 'final_nodes_with_ids.csv'.
Merged CSV 'cleaned_nodes.csv' and wrote results to 'final_nodes_with_ids.csv'.


In [15]:
if __name__ == "__main__":
    input_file = "final_nodes_with_ids.csv"      
    output_file = "final_nodes_with_ids_sorted.csv"    
    tn.sort_for_current_time(input_file, output_file)
    print("")
    print(f"Sorted CSV '{input_file}' and wrote results to '{output_file}'.")


Current Eorzean time (raw): 15:17

Active nodes (with time difference in minutes):
        ID   Time                   Item Name                   Location  \
5    33006  16:00               Rarefied Kelp                The Tempest   
4    33008  16:00              Rarefied Coral                The Tempest   
18   19857  16:00              Jhammel Ginger                  The Lochs   
19   32998  16:00  Rarefied Shiitake Mushroom               The Ruby Sea   
30   12898  16:00                       Morel  The Dravanian Hinterlands   
29   12897  16:00                Cloud Banana  The Dravanian Hinterlands   
78   32980  16:00    Rarefied Titancopper Ore                   Lakeland   
77   32982  16:00       Rarefied Raw Petalite                   Lakeland   
76   27728  16:00                Raw Lazurite     The Rak'tika Greatwood   
100  32970  16:00             Rarefied Pyrite    The Dravanian Forelands   
99   32971  16:00         Rarefied Chalcocite    The Dravanian Forelands   
87   

In [ ]:
from IPython.display import display
import pandas as pd
import os
from contextlib import redirect_stdout

if __name__ == "__main__":
    input_file = "final_nodes_with_ids_sorted.csv"
    output_file = "final_nodes_with_ids_market.csv"
    with open(os.devnull, 'w') as fnull:
        with redirect_stdout(fnull):
            tn.generate_market_data(input_file, output_file)
    print("")
    print(f"Generated market data for '{input_file}' and wrote results to '{output_file}'.")
    df_market = pd.read_csv("final_nodes_with_ids_market.csv")
    display(df_market)



Generated market data for 'final_nodes_with_ids_sorted.csv' and wrote results to 'final_nodes_with_ids_market.csv'.


,ID,Time,Item Name,Location,Coordinates,Parsed Time,time_diff,minListing_world,minListing_dc,recentPurchase_world,recentPurchase_dc,averageSalePrice_dc,dailySaleVelocity_dc
0,19857,16:00,Jhammel Ginger,The Lochs,"(x8,y8)",1900-01-01 16:00:00,43,499,2,75,75,86.528571,38.148282
1,12898,16:00,Morel,The Dravanian Hinterlands,"(x19,y36)",1900-01-01 16:00:00,43,30,10,43,33,117.533333,4.087313
2,12897,16:00,Cloud Banana,The Dravanian Hinterlands,"(x19,y36)",1900-01-01 16:00:00,43,300,30,900,487,487.000000,2.724873
3,27728,16:00,Raw Lazurite,The Rak'tika Greatwood,"(x19,y21)",1900-01-01 16:00:00,43,5,4,45,45,NaN,NaN
4,19971,16:00,Raw Azurite,Azim Steppe,"(x5,y29)",1900-01-01 16:00:00,43,194,194,300,248,313.060302,54.224903
5,10096,17:00,Rosemary,East Shroud,"(x22,y30)",1900-01-01 17:00:00,103,600,300,450,500,1032.971014,37.603172
